# Chapter_8  데이터 준비하기 : 조인(join), 병합(merge), 변형(pivot 등)

## 8.1 계층적 색인

계층적 색인은 축에 대해 다중 색인 단계를 지정할 수 있도록 해준다.

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.Series(np.random.randn(9),
                index = [['a','a','a','b','b','c','c','d','d'],
                        [1,2,3,1,3,1,2,2,3]])

data

a  1    0.248740
   2    1.108010
   3   -0.204819
b  1   -0.150055
   3   -0.845520
c  1   -0.800314
   2   -0.164278
d  2   -0.518101
   3    0.108904
dtype: float64

In [4]:
# 위 객체는 MultiIndex를 색인하는 Series 인데, 색인의 계층을 보여주고 있다.
# 바로 위 단계의 색인을 이용해서 하위 계층을 직접 접근할 수 있다.

data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [5]:
# 계층적으로 색인된 객체는 데이터의 부분집합을 부분적 색인으로 접근하는 것이 가능하다.

data['b']

1   -0.150055
3   -0.845520
dtype: float64

In [6]:
data['b':'c']

b  1   -0.150055
   3   -0.845520
c  1   -0.800314
   2   -0.164278
dtype: float64

In [7]:
data.loc[['b', 'd']]

b  1   -0.150055
   3   -0.845520
d  2   -0.518101
   3    0.108904
dtype: float64

In [8]:
# 하위 계층의 객체를 선택하는 것도 가능하다.

data.loc[:, 2]

a    1.108010
c   -0.164278
d   -0.518101
dtype: float64

In [9]:
# unstack 메서드를 사용하면 데이터를 새롭게 배열할 수도 있다.

data.unstack()

,1,2,3
a,0.248740,1.108010,-0.204819
b,-0.150055,NaN,-0.845520
c,-0.800314,-0.164278,NaN
d,NaN,-0.518101,0.108904


In [10]:
# unstack의 반대는 stack 메서드로 작업한다.

data.unstack().stack()

a  1    0.248740
   2    1.108010
   3   -0.204819
b  1   -0.150055
   3   -0.845520
c  1   -0.800314
   2   -0.164278
d  2   -0.518101
   3    0.108904
dtype: float64

In [15]:
# DataFrame 에서는 두 축 모두 계층적 색인을 가질 수 있다.

frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index=[['a','a','b','b'],[1,2,1,2]],
                    columns=[['순천', '순천', '여수'], ['파랑', '빨강', '초록']])

frame

순천      여수
    파랑  빨강  초록
a 1  0   1   2
  2  3   4   5
b 1  6   7   8
  2  9  10  11

In [16]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state     순천      여수
color     파랑  빨강  초록
key1 key2           
a    1     0   1   2
     2     3   4   5
b    1     6   7   8
     2     9  10  11

In [17]:
# 색인도 된다.

frame['순천']

color      파랑  빨강
key1 key2        
a    1      0   1
     2      3   4
b    1      6   7
     2      9  10

### 8.1.1 계층의 순서를 바꾸고 정렬하기

In [18]:
# swaplevel 은 넘겨받은 두 개의 계층 번호나 이름이 뒤바뀐 새로운 객체를 반환한다.

frame.swaplevel('key1', 'key2')

state     순천      여수
color     파랑  빨강  초록
key2 key1           
1    a     0   1   2
2    a     3   4   5
1    b     6   7   8
2    b     9  10  11

In [24]:
# sort_index 메서드는 단일 계층에 속한 데이터를 정렬한다.
# swaplevel 을 이용해서 계층을 바꿀 때 sort_index를 사용해서 결과가 사전적으로 정렬되도록 만드는 것도 드물지 않다.

frame.sort_index(level=1)

state     순천      여수
color     파랑  빨강  초록
key1 key2           
a    1     0   1   2
b    1     6   7   8
a    2     3   4   5
b    2     9  10  11

In [20]:
frame.swaplevel(0, 1).sort_index(level=0)

state     순천      여수
color     파랑  빨강  초록
key2 key1           
1    a     0   1   2
     b     6   7   8
2    a     3   4   5
     b     9  10  11

### 8.1.2 계층별 요약 통계

In [25]:
# level 옵션을 써보자.

frame.sum(level='key2')

state  순천      여수
color  파랑  빨강  초록
key2             
1       6   8  10
2      12  14  16

In [26]:
frame.sum(level='color', axis=1)

color      파랑  빨강  초록
key1 key2            
a    1      0   1   2
     2      3   4   5
b    1      6   7   8
     2      9  10  11

### 8.1.3 DataFrame의 컬럼 사용하기

In [27]:
# row 를 옮기거나 columns을 옮기거나

frame = pd.DataFrame({'a':range(7), 'b':range(7,0,-1),
                     'c':['one','one', 'one', 'two', 'two', 'two', 'two'],
                     'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [28]:
# DataFrame 의 set_index 함수는 하나 이상의 컬럼을 색인으로 하는 새로운 DataFrame 을 생성한다.

frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [29]:
# 명시적으로 컬럼을 남겨두지 않으면 삭제된다.

frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [31]:
# reset_index 는 set_index 와 반대 개념이다. 계층적 색인 단계가 column으로 이동한다.

frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 데이터 합치기

pandas 객체에 저장된 데이터는 여러 가지 방법으로 합칠 수 있다.

- pandas.merge는 하나 이상의 키를 기준으로 DataFrame 의 row를 합친다. SQL이나 다른 관계형 데이터 베이스의 join과 비슷하다.
- pandas.concat 은 하나의 축을 따라 객체를 이어 붙인다.
- combile_first 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다.

### 8.2.1 데이터베이스 스타일로 DataFrame 합치기

In [33]:
# 병합(merge)이나 조인(join) 연산은 관계형 DB의 핵심 연산이다. 하나 이상의 키를 사용해서 데이터 집합의 row를 합친다.


df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                   'data2': range(3)})

print(df1)
df2

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6


,key,data2
0,a,0
1,b,1
2,d,2


In [34]:
# 위 DF에 대해서 merge 를 써보자.

pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [35]:
# 어떤 column을 병합할 것인지 명시하지 않았다.
# merge 는 중복된 column 이름을 키로 사용한다.
# 명시적으로 지정해주자.

pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [36]:
# 만약 두 객체에 중복된 column이 하나도 없다면 따로 지정해주자.

df3 = pd.DataFrame({'lkey':['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1': range(7)})
df4 = pd.DataFrame({'rkey':['a', 'b', 'd'],
                   'data2': range(3)})

In [37]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [38]:
# 결과를 살펴보면 'c' 와 'd'에 해당하는 값이 빠진 것을 알 수 있다.
# merge 함수는 기본적으로 내부 조인(inner join) 을 수행하여 교집합인 결과를 반환한다.
# how 인자로 'left', 'right', 'outer' 를 넘겨서 각각 왼쪽 조인, 오른쪽 조인, 외부 조인을 수행할 수 있다.
# 외부 조인은 합집합, 왼쪽, 오른쪽은 각각의 로우를 포함하는 결과를 반환한다.

pd.merge(df1, df2, how= 'outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


#### how 옵션에 따른 join 연산

- 'inner' 양쪽 테이블 모두에 존재하는 키 조합을 사용
- 'left' 왼쪽 테이블
- 'right' 오른쪽 테이블
- 'output' 양쪽 테이블에 존재하는 모든 키 조합을 사용

In [39]:
# 다대다 병합은 잘 정의되어 있긴 하지만 직관적이지는 않다.

df1 = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df2 = pd.DataFrame({'key':['a', 'b', 'a', 'b', 'd'],
                   'data2': range(5)})

In [40]:
print(df1)
df2

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5


,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [41]:
# 다대다 조인은 두 로우의 데카르트곱을 반환한다.
# 왼쪽DF에 3개의 'b' 가 있고, 오른쪽에 2개가 있으면 총 6개의 'b' 로우가 된다.

pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [42]:
# join 메서드는 결과에 나타나는 구별되는 키에 대해서만 적용된다.

pd.merge(df1, df2, how='inner') # inner는 양쪽 모두에게 존재하는 키를 반환

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [43]:
# 여러 개의 키를 병합하려면 column 이름이 담긴 list 를 넘기면 된다.

left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                    'key2': ['one', 'two', 'one'],
                    'lval': [1, 2, 3]})
right = pd.DataFrame({'key1':['foo', 'foo', 'bar', 'bar'],
                     'key2': ['one', 'one', 'one', 'two'],
                     'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how= 'outer') # outer 는 양쪽의 모든 키 반환

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [44]:
# 마지막으로 겹치는 column 이름에 대한 처리다.
# 축 이름을 변경해서 수동으로 이름이 겹치게 할 수도 있고,
# merge 함수에 있는 suffixes 인자로 두 DF 객체에서 겹치는 column 이름 뒤에 붙일 문자열을 지정해줄 수 있다.

pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [45]:
pd.merge(left, right, on='key1', suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### merge 함수 인자 목록

- left : 왼쪽 DF
- right : 오른쪽
- how : 'inner', 'outer', 'left', 'right', 기본값은 'inner'
- on : 조인하려는 column 이름. 반드시 두 DF 객체 모두에 존재하는 이름이어야.
- left_on : 조인키로 사용할 left DataFrame의 column
- right_on:
- left_index : 조인키로 사용할 left DataFrame 의 색인 row( 다중 색인일 경우 키)
- right_index
- sort : 조인키에 따라 병합된 데이터를 사전순으로 정렬, 기본값은 True
- suffixes : column 이름이 겹칠 경우 각 column 뒤에 붙일 문자열의 튜플. 기본값은 ('_x', '_y')
- copy : False 일 경우, 예외적인 경우에 데이터가 결과로 복사되지 않도록 함. 기본값은 항상 복사가 이루어진다.
- indicator : merge 라는 이름의 특별한 column 을 추가하여 각 row 의 소스가 어디인지 나타낸다. 'left_only', 'right_only', 'both' 값을 가짐

### 8.2.2 색인 병합하기

In [46]:
# 병합하려는 키가 DataFrame의 색인일 경우가 있다.
# 이런 경우에는 left_index=True 혹은 right_index=Ture 옵션을 지정해서 해당 색인을 병합키로 사용할 수 있다.

left1 = pd.DataFrame({'key':['a', 'b', 'a', 'a', 'b', 'c'],
                     'value': range(6)})
right1 = pd.DataFrame({'group_val':[3.5, 7]}, index=['a', 'b'])

print(left1)
right1

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5


,group_val
a,3.5
b,7.0


In [47]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [48]:
# 외부조인을 실행해서 합집합을 구할 수 있다.

pd.merge(left1, right1, left_on = 'key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [50]:
# 계층 색인된 데이터는 암묵적으로 여러 키를 병합하는 것이라 약간 복잡하다.

lefth = pd.DataFrame({'key1': ['순천', '순천', '순천', '여수', '여수'],
                     'key2':[2000,2001,2002, 2001, 2002],
                     'data':np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6,2)),
                     index = [['여수', '여수', '순천', '순천', '순천', '순천'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                     columns = ['event1', 'event2'])

print(lefth)
righth

  key1  key2  data
0   순천  2000   0.0
1   순천  2001   1.0
2   순천  2002   2.0
3   여수  2001   3.0
4   여수  2002   4.0


event1  event2
여수 2001       0       1
   2000       2       3
순천 2000       4       5
   2000       6       7
   2001       8       9
   2002      10      11

In [53]:
# 이 경우에는 list 로 여러 개의 column을 지정해서 병합해야 한다.

pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,순천,2000,0.0,4,5
0,순천,2000,0.0,6,7
1,순천,2001,1.0,8,9
2,순천,2002,2.0,10,11
3,여수,2001,3.0,0,1


In [54]:
pd.merge(lefth, righth, left_on = ['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,순천,2000,0.0,4.0,5.0
0,순천,2000,0.0,6.0,7.0
1,순천,2001,1.0,8.0,9.0
2,순천,2002,2.0,10.0,11.0
3,여수,2001,3.0,0.0,1.0
4,여수,2002,4.0,NaN,NaN
4,여수,2000,NaN,2.0,3.0


In [55]:
# 양쪽에 공통적으로 존재하는 여러 개의 색인을 병합하는 것도 가능하다.

left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                    index = ['a', 'c', 'e'],
                    columns = ['순천', '여수'])
right2 = pd.DataFrame([[7., 8], [9., 10], [11., 12.], [13, 14]],
                     index=['b', 'c', 'd', 'e'],
                     columns = ['목포', '광주'])

In [56]:
print(left2)
right2

    순천   여수
a  1.0  2.0
c  3.0  4.0
e  5.0  6.0


,목포,광주
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [57]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,순천,여수,목포,광주
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [58]:
# 색인으로 병합할 때 DataFrame의 join 메서드를 사용하면 편하다.
# join 메서드는 column 이 겹치지 않으며 완전히 같거나 유사한 색인 구조를 가진 여러 개의 DataFrame 객체를 병합할 때 사용한다.

left2.join(right2, how='outer')

,순천,여수,목포,광주
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [59]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [60]:
# 색인 대 색인으로 두 DF 를 병합하려면 그냥 간단하게 join 메서드로 넘긴다.
# 하지만 보통 이런 병합은 concat을 사용한다.

another = pd.DataFrame([[7., 8], [9., 10], [11., 12.], [16, 17]],
                     index=['a', 'c', 'e', 'f'],
                     columns = ['서울', '원주'])
another

,서울,원주
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [61]:
left2.join([right2, another])

,순천,여수,목포,광주,서울,원주
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [63]:
left2.join([right2, another], how='outer')

C:\Users\thinp\Anaconda3\lib\site-packages\pandas\core\frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,순천,여수,목포,광주,서울,원주
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


### 8.2.3 축 따라 이어붙이기

In [65]:
# 데이터를 합치는 또 다른 방법으로는 이어붙이기(concatenation)(연결(binding), 적층(stacking)이라고도 함)이 있다.

arr = np.arange(12).reshape((3,4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [66]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [67]:
# pandas 에서는 concat 함수가 있다.
# 색인이 겹치지 않는 3개의 Series 객체가 있다고 하자.

s1 = pd.Series([0,1], index = ['a', 'b'])
s2 = pd.Series([2, 3, 4], index = ['c', 'd', 'e'])
s3 = pd.Series([5, 6], index = ['f', 'g'])

In [68]:
# 이 세 객체를 list로 묶어서 concat 함수에 전달하면 값과 색인을 연결해준다.

pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [69]:
# concat 함수는 axis=0 을 기본값으로 하여 새로운 Series 객체를 생성한다.
# 만약 axis=1 을 넘긴다면 결과는 Series가 아니라 DataFrame 이 될 것이다.

pd.concat([s1, s2, s3], axis=1)

C:\Users\thinp\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [71]:
# 겹치는 축이 없기 때문에 외부 조인으로 정렬된 합집합을 얻었지만 join='inner'를 넘겨서 교집합을 구할 수 있다.

s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [72]:
pd.concat([s1, s4], axis=1)

C:\Users\thinp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [73]:
# f 라벨과 g 라벨은 inner 옵션으로 인해 사라진다.

pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [74]:
# join_axes 인자로 병합하려는 축을 직접 지정해줄 수도 있다.

pd.concat([s1, s4], axis=1, join_axes=[['a', 'b', 'b', 'e']])

,0,1
a,0.0,0.0
b,1.0,1.0
b,1.0,1.0
e,NaN,NaN


In [75]:
# 계층적 색인을 생성하여 이어붙인 축에 대해 식별이 가능하도록 할 수 있다.
# keys 인자를 이용한다.

result = pd.concat([s1, s1, s3], keys= ['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [76]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [77]:
# Series 를 axis=1 로 병합할 경우 keys는 DataFrame 의 column 제목이 된다.

pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

C:\Users\thinp\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [78]:
# DataFrame 객체에 대해서도 지금까지와 같은 방식으로 적용할 수 있다.

df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                  columns= ['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                  columns= ['three', 'four'])
print(df1)
df2

   one  two
a    0    1
b    2    3
c    4    5


,three,four
a,5,6
c,7,8


In [79]:
pd.concat([df1, df2], axis=1, keys= ['level1', 'level2'])

C:\Users\thinp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [80]:
# list 대신 dic 을 넘기면 dic 의 키가 keys 옵션으로 사용된다.

pd.concat({'level1':df1, 'level2':df2}, axis=1)

C:\Users\thinp\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [81]:
# 새로 생성된 계층의 이름은 names 인자로 지정할 수 있다.

pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
         names=['upper', 'lower'])

C:\Users\thinp\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [82]:
# DataFrame 의 row 색인이 분석에 필요한 데이터를 포함하고 있지 않은 경우에는?!

df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

print(df1)
df2

          a         b         c         d
0  1.178069  0.658516 -0.024700  0.431973
1  0.615522  1.088092 -0.743481 -1.638279
2 -0.286852  0.030107  0.348671  0.327796


,b,d,a
0,-0.083624,-0.507779,-0.114154
1,1.173295,-0.415937,-1.365307


In [83]:
# 이럴 경우는 ignore_index = True 옵션을 주면 된다.

pd.concat([df1, df2], ignore_index=True)

C:\Users\thinp\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,a,b,c,d
0,1.178069,0.658516,-0.024700,0.431973
1,0.615522,1.088092,-0.743481,-1.638279
2,-0.286852,0.030107,0.348671,0.327796
3,-0.114154,-0.083624,NaN,-0.507779
4,-1.365307,1.173295,NaN,-0.415937


#### concat 함수 인자

- objs : 이어붙일 pandas 객체의 dic이나 list. 필수 인자
- axis : 축.
- join : join방식, inner, outer. 기본값은 outer
- join_axes : 합집합/교집합을 수행하는 대신 다른 n-1 축으로 사용할 색인을 지정한다.
- keys : 이어붙일 객체나 이어붙인 축에 대한 계층 색인을 생성하는 데 연관된 값.
- levels : 계층 색인 레벨로 사용할 색인을 지정. keys 가 넘어온 경우 여러 개의 색인을 지정
- names : keys나 levels 혹은 둘 다 있을 경우 생성된 계층 레벨을 위한 이름
- verify_integrity : 이어붙인 객체에 중복되는 축이 있는지 검사하고 있다면 예외를 발생. 기본값은 False 로 중복 허용
- ignore_index : 이어붙인 축의 색인을 유지하지 않고 range(total_length)로 새로운 색인을 생성한다.

### 8.2.4 겹치는 데이터 합치기

In [84]:
# 데이터를 합칠 때 병합이나 이어붙이기로는 불가능한 상황이 있다.
# 두 데이터셋의 색인이 일부 겹치거나 전체가 겹치는 경우가 그렇다.
# 벡터화된 if-else 구문을 표현하는 Numpy의 where 함수를 알아보자.

a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
             index= ['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
             index= ['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

print(a)
b

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64


f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [85]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [86]:
# Series 객체의 combine_first 메서드는 위와 동일한 연산을 제공하며 데이터 정렬 기능까지 제공한다.

b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [87]:
# DataFrame 에서 combine_first 메서드는 column 에 대해 같은 동작을 한다.

df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                   'b': [np.nan, 2., np.nan, 6],
                   'c': range(2, 18, 4)})

df2 =pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                  'b': [np.nan, 3., 4., 6., 8.]})

print(df1)
df2

     a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14


,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [88]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3 재형성과 피벗

표 형식의 데이터를 재배치하는 다양한 연산이 존재한다. 이런 연산을 재형성 또는 피벗 연산이라고 한다.

### 8.3.1 계층적 색인으로 재형성하기

계층적 색인은 DataFrame의 데이터를 재배치하는 다음과 같은 방식을 제공한다.

 - stack : data의 column 을 row 로 pivot(또는 회전) 시킨다.
 
 - unstack : row를 column으로 pivot 시킨다.

In [92]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                   index = pd.Index(['순천', '여수'], name= 'state'),
                   columns = pd.Index(['one', 'two', 'three'], name = 'number'))
data

number,one,two,three
state,,,
순천,0,1,2
여수,3,4,5


In [93]:
# stack 메서드를 사용하면 column 이 row 로 pivot 되면서 다음과 같은 Series 객체를 반환

result = data.stack()
result

state  number
순천     one       0
       two       1
       three     2
여수     one       3
       two       4
       three     5
dtype: int32

In [94]:
# unstack 메서드를 사용하면 다시 DataFrame 으로 가져온다.

result.unstack()

number,one,two,three
state,,,
순천,0,1,2
여수,3,4,5


In [95]:
# 기본적으로 가장 안쪽에 있는 level부터 끄집어 내는데, level숫자나 이름을 전달해서 끄집어낼 단계를 지정할 수도 있다.

result.unstack(0)

state,순천,여수
number,,
one,0,3
two,1,4
three,2,5


In [96]:
result.unstack('state')

state,순천,여수
number,,
one,0,3
two,1,4
three,2,5


In [97]:
# 해당 레벨에 있는 모든 값이 하위 그룹에 속하지 않을 경우 unstack 을 하게 되면 누락된 데이터가 생길 수 있다.

s1 = pd.Series([0, 1, 2, 3], index = ['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index = ['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [98]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [99]:
# stack 메서드는 누락된 데이터를 자동으로 걸러내기 때문에 연산을 쉽게 원상 복구할 수 있다.

data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [100]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [101]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [102]:
# DataFrame 을 unstack() 할 때 unstack 레벨은 결과에서 가장 낮은 단계가 된다.

df = pd.DataFrame({'left': result, 'right':result+5},
                 columns=pd.Index(['left', 'right'], name='side'))

df

side          left  right
state number             
순천    one        0      5
      two        1      6
      three      2      7
여수    one        3      8
      two        4      9
      three      5     10

In [103]:
df.unstack('state')

side   left    right    
state    순천 여수    순천  여수
number                  
one       0  3     5   8
two       1  4     6   9
three     2  5     7  10

In [104]:
# stack 을 호출할 때 쌓을 축의 이름을 정할 수 있다.

df.unstack('state').stack('side')

state         순천  여수
number side         
one    left    0   3
       right   5   8
two    left    1   4
       right   6   9
three  left    2   5
       right   7  10

### 8.3.2 긴 형식에서 넓은 형식으로 피벗하기

In [ ]:
# 이 부분은 예제가 필요하므로 책으로 읽기로 하자.

